In [ ]:
##########################################################################################################
# Generation d'un fichier html des revues non valides saisies dans HAL pour une collection en particulier
# Au moment du dépôt dans HAL, il arrive que l'on saisisse un nom de revue qui n'est pas reconnu dans la référentiel Aurehal
# ce titre est donc créé dans Aurehal avec le statut "non valide" et doit être validé par metadata stewards de HAL.
# Ces revues ont le statut "INCOMING" dans Aurehal
# Le présent script permet de repérer ces revues saisies dans les publications d'une collection.
# Il suffit de remplacer le nom de la collection entre guillemets dans la permière variable "collection" ci-dessous
# Cela crée un fichier html autonome. 
# Attention, ce fichier ne se met pas à jour seul, il faut relancer le script pour obtenir une nouvelle page à jour.
# Les colonnes du fichier html sont triables
##########################################################################################################
## Code créé par Kumar Guha pour INRIA, sept 2024.
##########################################################################################################

# importation des "bibliothèques" nécessaires à l'exécution du script
import requests
import re
import os
import sys
from urllib.parse import quote
from datetime import datetime

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# Définition de la collection HAL où l'on recherche les titres non valides    #
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
collection = "INRIA"


## Spécifier le répertoire où s'enreistrera le fichier contenant le résultat
repertoire_cible = 'Revues_Incoming/' # ce répertoire sera un sous-répertoire de celui où se trouve le script
## Créer le répertoire s'il n'existe pas
os.makedirs(repertoire_cible, exist_ok=True)

# Définition de la date du jour
date_fichier = str(datetime.now().strftime("%Y-%m-%d"))
current_date = datetime.now().strftime("%d/%m/%Y")

# Redirection de la sortie standard vers un fichier html : tous les "print" seront enregistrés dans le fichier au lieu d'apparaître dans l'écran de résultat
fichier_sortie = os.path.join(repertoire_cible, 'journal_log_revues_incoming_tri_{}_{}.html'.format(collection, date_fichier))
sys.stdout = open(fichier_sortie, 'w')

#####################################################################################
## Préparation
## Fonctions utilisées par le script
#####################################################################################

# Fonction pour vérifier que l'url définie plus bas renvoie une réponse ou non
def get_api_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print("Erreur lors de la requête à l'API :", response.status_code)
        return None

# Liste de mots vides en anglais et en français (utilisée dans la fontion "clean_title")
stop_words = set([
    "the", "of", "and", "to", "in", "a", "is", "for", "on", "that", "with", "by",
    "sont", "les", "des", "et", "pour", "dans", "une", "est", "par", "avec", "qui", 
])

# Fonction qui nettoie le titre de la revue pour permettre de faire un lien vers la recherche dans Aurehal sur ce titre "appauvri"
def clean_title(title):
    # Expression régulière pour capturer les mots significatifs
    pattern = re.compile(r'\b[\w\'-]+\b') # modèle de mots complets avec lettres, apostrophes, et "-"
    
    # Utiliser le "pattern" pour extraire les mots du titre
    words = pattern.findall(title)

    # Filtrer les mots vides (on supprime tous ce qui est dans "stop words")
    meaningful_words = [word for word in words if word.lower() not in stop_words]

    # Rejoindre les mots significatifs en une seule chaînej séparés par un signe +
    cleaned_title = '+'.join(meaningful_words)
    
    return cleaned_title

# Initialisation de la liste pour stocker les données au format JSON (optionnel)
output_data = []


#################################################################################################
## Début du script de recherche des revues "Incoming" (non valides)
#################################################################################################

# On définit l'api de recherche des revues Incoming dans une variable "url1"
# Critères de l'api:
# collection : défini en premier et à part, pour pouvoir changer la collection cible
# journalValid_s:INCOMING (statut de la revue, incoming=non valide)
# docType_s:ART (type de doc : article)
# rows=1000 (limité à 1000 réponses)
# sort= submittedDate_s desc (critère de tri : date de soumission décroissante)
# fl=journalTitle_s,uri_s,journalId_i,halId_s,submittedDate_s (champs à récupérer, on utilisera ces métadonnées)
# (en langage clair fl=titre revue, url notice hal, id aurehal revue, id notice Hal, date soumission)


# réponse attendue : liste des notices concernées et, 
# pour chaque notice, la réponse donne la valeur des champs journalTitle_s (Nom de la revue),uri_s (url),journalId_i (ID de la revue),halId_s (ID de la notice),submittedDate_s (date de dépôt)
url1 = f"https://api.archives-ouvertes.fr/search/{collection}?q=journalValid_s:INCOMING AND docType_s:ART&rows=1000&sort=submittedDate_s%20desc&fl=journalTitle_s,uri_s,journalId_i,halId_s,submittedDate_s"

# On envoie url1 à la fonction "get_api_data" pour voir s'il y a une réponse 
# et récupérer les données de la réponse (dans ce cas, "data1" existe)
data1 = get_api_data(url1)

# Si data1 existe = si l'api a donné une réponse, on va récupérer des informations
# et les afficher dans un tableau html et les stocker dans un fichier json
if data1:
    # On place dans une variable les résultats, composés d'une série de "docs" (= notices hal)
    results = data1.get('response', {}).get('docs', [])
    
    # Récupérer les noms uniques de journalTitle_s pour compter le nombre de revues non valides
    unique_journal_titles = set(result.get('journalTitle_s', '') for result in results if result.get('journalTitle_s'))

    # On affiche le nombre de titres non valides et le nombre de notices concernées
    print(f"{len(unique_journal_titles)} titres non valides dans {len(results)} notices HAL (consulté le : {current_date})")
    
    # On affiche en html un lien vers l'api
    print(f"<a href='{url1}' target='_blank'>Lancer api</a>")
    print(f"<p>Vos corrections seront prises en compte par l'API à la prochaine indexation des notices corrigées (env. 24h de délai)</p>")
    
    # On explique que les colonnes sont triables
    print("<p><i>Cliquez sur le titre de chaque colonne pour la trier</i></p>")
    
    # Début du tableau HTML
    print("<table id='journal_table' style='text-align: left; margin: 1px;'>")
    print("<thead><tr><th id='journal_title'>Titre de la revue</th><th id='notice_hal'>Notice HAL</th><th id='Date_soumission_clean'>Date soumission</th><th id='journal_id'>ID Aurehal</th><th id='alternative_valide'>Alternative valide ?</th></tr></thead>")
    print("<tbody>")
    
    #Boucle "for": pour chaque notice résultants de l'api, 
    #on récupère les informations qui servent au tableau et au fichier json
    for result in results:
        # On met les informations des champs du résultat dans des variables
        journal_title = result.get('journalTitle_s', '') # titre de la revue
        notice_hal = result.get('uri_s', '') # url de la notice Hal
        halId_ref = result.get('halId_s', '') # hal ID (hal-xxx) de la notice, il sera cliquable avec l'url
        journalId_incoming = result.get('journalId_i', '') # identifiant Aurehal de la revue
        Date_soumission = result.get('submittedDate_s', '') # date soumission de la notice Hal
        cleaned_journal_title = clean_title(journal_title) # titre "appauvri" de la revue (les mots séparés par +)
        
        # On définit des variables contenant des url vers Aurehal
            # url menant à la revue dans Aurehal :
        journalId_incoming_in_Aurehal = f"https://aurehal.archives-ouvertes.fr/journal/browse?critere={journalId_incoming}" 
            # url de recherche sur le titre appauvri dans Aurehal :
        search_title_in_Aurehal = f"https://aurehal.archives-ouvertes.fr/journal/browse?critere={quote(str(cleaned_journal_title))}"

        #formatage de la date de soumission au format jj-mm-aaaa
        Date_complete = datetime.strptime(Date_soumission, "%Y-%m-%d %H:%M:%S")
        Date_soumission_clean = Date_complete.strftime("%d-%m-%Y")
        Date_soumission_yyyymmjj = Date_complete.strftime("%Y-%m-%d")
        
        # On va rechercher le titre "appauvri" dans Aurehal.
        # S'il y a un titre valide dans le résultat, on l'indiquera comme potentiel titre valide pouvant remplacer le titre non valide dans la liste des revues.
        if journal_title:
            url2 = "https://api.archives-ouvertes.fr/ref/journal/?q=text:{}&fl=docid,valid_s,title_s".format(cleaned_journal_title)  
            data2 = get_api_data(url2)
            
            # On initialise une variable qui changera de valeur en fonction du résultat de l'url2
            valid_status = "non défini"
            
            # Si l'url2 donne un résultat
            if data2:
                
                # Vérifier si la clé "response" est présente dans les données
                if 'response' in data2:
                    
                    # Accéder à la liste de titres Aurehal répondant à la requête
                    resultats = data2['response'].get('docs', [])
                    
                    # Dernier contrôle qu'il y a bien un résultat
                    if resultats:
                        # Si dans les résultats, on trouve un champ "valid_s = VALID"
                        if any(resultat.get('valid_s', '') == "VALID" for resultat in resultats):
                            # On renseigne la variable "valid_status" qui sera intégrée au tableau html
                            valid_status = f"<a href='{search_title_in_Aurehal}' target='_blank'>Il y a peut-être un titre identique valide</a>"
                            
                            # On renseigne une variable qui sera intégrée au fichier json
                            alternative_valide = "True"
                            
                            #On crée une liste de toutes les données de ce titre pour le fichier json
                            entry = {
                                "date": current_date,
                                "journal_title": journal_title,
                                "cleaned_journal_title": cleaned_journal_title,
                                "url_notice_hal": notice_hal,
                                "ref_notice_hal": halId_ref,
                                "ref journal non valide": journalId_incoming,
                                "alternative_valide": alternative_valide,
                                "lien aurehal journal non valide": journalId_incoming_in_Aurehal,
                                "Recherche ce titre dans Aurehal": search_title_in_Aurehal,
                                "Date depot hal": Date_soumission_yyyymmjj,
                            }
                            
                            #On intègre cette liste "entry" dans une collection virtuelle "output_data"
                            output_data.append(entry)
                        
                        # Si la condition valid_status = VALID n'est pas vraie,
                        else:
                            # On renseigne la variable "valid_status" qui sera intégrée au tableau html
                            valid_status = f"Pas de titre valide trouvé dans Auréhal. <a href='{search_title_in_Aurehal}' target='_blank'>Vérifier.</a>"
                            
                            # On renseigne une variable qui sera intégrée au fichier json
                            alternative_valide = "False"
                            
                            #On crée une liste de toutes les données de ce titre pour le fichier json
                            entry = {
                                "date": current_date,
                                "journal_title": journal_title,
                                "cleaned_journal_title": cleaned_journal_title,
                                "notice_hal": notice_hal,
                                "ref_notice_hal": halId_ref,
                                "ref journal non valide": journalId_incoming,
                                "alternative_valide": alternative_valide,
                                "lien aurehal journal non valide": journalId_incoming_in_Aurehal,
                                "Recherche ce titre dans Aurehal": search_title_in_Aurehal,
                                "Date depot hal": Date_soumission_yyyymmjj,
                            }
                            #On intègre cette liste "entry" dans une collection virtuelle "output_data"
                            output_data.append(entry)
            else:
                print("pas de reponse <br />")
            # Ecriture du fichier html avec des identifiants pour le javascript qui permettra de trier par colonne
            print(f"<tr data-journal_title='{journal_title}' data-notice_hal='{notice_hal}'  data-Date_soumission_clean='{Date_soumission_clean}' data-journal_id='{journalId_incoming}' data-alternative_valide='{alternative_valide}' >")
            print(f"<td style='border-bottom: 1px solid black;'>{journal_title}</td><td style='border-bottom: 1px solid black;'><a href='{notice_hal}' target='_blank'>{halId_ref}</a></td><td style='border-bottom: 1px solid black;'>{Date_soumission_clean}</td><td style='border-bottom: 1px solid black;'><a href='{journalId_incoming_in_Aurehal}' target='_blank'>{journalId_incoming}</a></td><td style='border-bottom: 1px solid black;'>{valid_status}</td>")
            print("</tr>")

        # au cas très improbable où aucun titre ne revient de la requête
        else:
            print("<tr><td colspan='5'>Erreur: Pas de titre</td></tr>")
    
    # Dernières balises de la page html
    print("</tbody></table>")

else:
    print("pas de réponse à l'api")



# Script JavaScript pour trier le tableau html par colonne
print("""
<script>
document.addEventListener('DOMContentLoaded', function() {
    const headers = document.querySelectorAll('#journal_table th');
    headers.forEach(header => {
        header.addEventListener('click', () => {
            const column = header.id;
            const table = header.closest('table');
            const rows = Array.from(table.querySelectorAll('tbody tr'));

            const direction = header.dataset.sortDirection || 'asc';
            const multiplier = direction === 'asc' ? 1 : -1;

            const sortedRows = rows.sort((a, b) => {
                const aValue = a.dataset[column];
                const bValue = b.dataset[column];
                // Si la colonne est la date, convertir en objet Date pour une comparaison correcte
                if (column === 'Date_soumission_clean') {
                    return new Date(aValue) > new Date(bValue) ? multiplier : -multiplier;
                }
                // Pour les autres colonnes,  comparaison normale
                return aValue.localeCompare(bValue) * multiplier;
            });

            table.querySelector('tbody').innerHTML = '';
            sortedRows.forEach(row => {
                table.querySelector('tbody').appendChild(row);
            });

            header.dataset.sortDirection = direction === 'asc' ? 'desc' : 'asc';
        });
    });
});
</script>
""")

# # Optionnel : Écrire les données de la collection "output_data" dans un fichier JSON
# with open('../Resultats/journal_log_revues_incoming.json', 'w') as json_file:
#     json.dump(output_data, json_file, indent=4)

# Réinitialisation de la sortie standard (les prints qui suivront s'afficheront à l'écran)
sys.stdout = sys.__stdout__

#
# Fin du script
#

JSONDecodeError: Expecting value: line 1 column 1 (char 0)